In [1]:
import gzip
import pickle

import mlflow
import mlflow.keras
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import (Conv1D, Embedding, GlobalMaxPooling1D,
                                    LSTM, Dense, Dropout, Bidirectional)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer



In [2]:
mlflow.set_tracking_uri('https://mlflow-service-1073438601911.us-west2.run.app')
mlflow.set_experiment('intern-question/project/final/mlflow')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1727047711746, experiment_id='1', last_update_time=1727047711746, lifecycle_stage='active', name='intern-question/project/final/mlflow', tags={}>

In [3]:
with open('/Users/lancesanterre/pipeline_edu/data/processed/pipeline_and_data.pkl', 'rb') as f:
    df = pd.read_pickle(f)
print(df.head())
# Assuming the DataFrame `df` has at least one column and you want to access the first column
questions = df['question']  # or df[0] if it's a Series or DataFrame
questions 

                                            question           labels
0  Which NFL team represented the AFC at Super Bo...  [1.0, 0.0, 0.0]
1  Which NFL team represented the NFC at Super Bo...  [1.0, 0.0, 0.0]
2                Where did Super Bowl 50 take place?  [1.0, 0.0, 0.0]
3                  Which NFL team won Super Bowl 50?  [1.0, 0.0, 0.0]
4  What color was used to emphasize the 50th anni...  [1.0, 0.0, 0.0]


0         Which NFL team represented the AFC at Super Bo...
1         Which NFL team represented the NFC at Super Bo...
2                       Where did Super Bowl 50 take place?
3                         Which NFL team won Super Bowl 50?
4         What color was used to emphasize the 50th anni...
                                ...                        
414915    How many keywords are there in the Racket prog...
414916            Do you believe there is life after death?
414917                                    What is one coin?
414918    What is the approx annual cost of living while...
414919                What is like to have sex with cousin?
Name: question, Length: 414920, dtype: object

In [4]:
filtered_labels = df['labels']
filtered_questions = questions 

# Simple LSTM

In [21]:
best_accuracy = 0
best_model_info = None

for input_dim in [1000, 2000, 4000, 5000]:
    for output_dim in [16, 32, 64, 128]:
        input_length = 10
        with mlflow.start_run():
            # Tokenization and Padding
            tokenizer = Tokenizer(num_words=input_dim)  # Adjust vocabulary size
            tokenizer.fit_on_texts(filtered_questions)
            sequences = tokenizer.texts_to_sequences(filtered_questions)
            X = pad_sequences(sequences, maxlen=input_length)

            # Convert labels to numpy array
            y = np.array(filtered_labels.tolist())

            # Model: Simple LSTM
            model_name = f"Simple_LSTM_{input_dim}_{output_dim}"
            model = Sequential([
                Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
                LSTM(output_dim),
                Dropout(0.5),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
            ])

            # Compile and train
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

            # Evaluate and log
            loss, accuracy = model.evaluate(X_test, y_test)
            mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
            mlflow.log_metrics({"loss": loss, "accuracy": accuracy})

            # Save the model with a unique name
            mlflow.keras.log_model(model, artifact_path=model_name)
            print(f"Model saved as {model_name} with accuracy: {accuracy:.2f}")

            # Check if this is the best model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model_info = {
                    "model_name": model_name,
                    "run_id": mlflow.active_run().info.run_id,
                }

            # End the current MLflow run to prepare for the next iteration
            mlflow.end_run()

# Register the best model
if best_model_info:
    print(f"Registering the best model: {best_model_info['model_name']} with accuracy: {best_accuracy:.2f}")
    mlflow.register_model(
        model_uri=f"runs:/{best_model_info['run_id']}/{best_model_info['model_name']}",
        name="Best_Simple_LSTM_Model"
    )


Epoch 1/10


/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10373/10373 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.8708 - loss: 0.3292 - val_accuracy: 0.9281 - val_loss: 0.1808
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9274 - loss: 0.1864 - val_accuracy: 0.9334 - val_loss: 0.1715
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9315 - loss: 0.1721 - val_accuracy: 0.9362 - val_loss: 0.1629
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9354 - loss: 0.1643 - val_accuracy: 0.9364 - val_loss: 0.1629
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9379 - loss: 0.1576 - val_accuracy: 0.9392 - val_loss: 0.1569
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9399 - loss: 0.1527 - val_accuracy: 0.9393 - val_loss: 0.1609
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9422 - loss: 0.1470 - val_accuracy: 0.9412 - val_loss: 0.1567
Epoch 8/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9438 - lo

2024/09/22 16:31:20 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/09/22 16:31:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 16:31:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run glamorous-shad-157 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/6d6ae46cd3a449e7ab19ba568a009c12.
2024/09/22 16:31:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.r

Model saved as Simple_LSTM_1000_16 with accuracy: 0.94
Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.8872 - loss: 0.2854 - val_accuracy: 0.9308 - val_loss: 0.1708
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9321 - loss: 0.1712 - val_accuracy: 0.9371 - val_loss: 0.1570
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9373 - loss: 0.1577 - val_accuracy: 0.9399 - val_loss: 0.1514
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9418 - loss: 0.1481 - val_accuracy: 0.9411 - val_loss: 0.1489
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9460 - loss: 0.1392 - val_accuracy: 0.9424 - val_loss: 0.1483
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9479 - loss: 0.1340 - val_accuracy: 0.9422 - val_loss: 0.1466
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9507 - loss: 0.1280 - val_accuracy: 0.9434 - val_loss: 0.1450
Epoch 8/10
103

...

2024/09/22 18:03:48 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 18:04:05 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 18:04:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run trusting-mule-681 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/8c7dbf0bdd464701b64e393be65473f2.
2024/09/22 18:04:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


Model saved as Simple_LSTM_5000_64 with accuracy: 0.92
Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 78s 7ms/step - accuracy: 0.8722 - loss: 0.3152 - val_accuracy: 0.9114 - val_loss: 0.2142
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9205 - loss: 0.1975 - val_accuracy: 0.9192 - val_loss: 0.2010
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.9341 - loss: 0.1661 - val_accuracy: 0.9208 - val_loss: 0.1964
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 79s 8ms/step - accuracy: 0.9440 - loss: 0.1438 - val_accuracy: 0.9243 - val_loss: 0.2055
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9531 - loss: 0.1221 - val_accuracy: 0.9229 - val_loss: 0.2114
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 79s 8ms/step - accuracy: 0.9604 - loss: 0.1037 - val_accuracy: 0.9248 - val_loss: 0.2211
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 78s 8ms/step - accuracy: 0.9673 - loss: 0.0873 - val_accuracy: 0.9250 - val_loss: 0.2295
Epoch 8/10
103

2024/09/22 18:17:26 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 18:18:12 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 18:18:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run colorful-shrew-603 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/c7140f0e6e17481395d86305c40559ce.
2024/09/22 18:18:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


Model saved as Simple_LSTM_5000_128 with accuracy: 0.93
Registering the best model: Simple_LSTM_1000_128 with accuracy: 0.95


Successfully registered model 'Best_Simple_LSTM_Model'.
2024/09/22 18:18:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best_Simple_LSTM_Model, version 1
Created version '1' of model 'Best_Simple_LSTM_Model'.


#  Bi-directional LSTM

In [22]:
for input_dim in [1000, 2000, 4000, 5000]:
    for output_dim in [16, 32, 64, 128]:
        input_length = 10
        with mlflow.start_run():
            # Tokenization and Padding
            tokenizer = Tokenizer(num_words=input_dim)
            tokenizer.fit_on_texts(filtered_questions)
            sequences = tokenizer.texts_to_sequences(filtered_questions)
            X = pad_sequences(sequences, maxlen=input_length)
            y = np.array(filtered_labels.tolist())

            # Model 2: Bi-directional LSTM
            model_name = f"Bidirectional_LSTM_{input_dim}_{output_dim}"
            model = Sequential([
                Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
                Bidirectional(LSTM(output_dim)),
                Dropout(0.5),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
            ])

            # Compile and train
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

            # Evaluate and log
            loss, accuracy = model.evaluate(X_test, y_test)
            mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
            mlflow.log_metric("loss", loss)
            mlflow.log_metric("accuracy", accuracy)

            # Save the model with a unique name
            mlflow.keras.log_model(model, artifact_path=model_name)
            print(f"Model saved as {model_name} with accuracy: {accuracy:.2f}")
        mlflow.end_run()


Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.8889 - loss: 0.2822 - val_accuracy: 0.9315 - val_loss: 0.1724
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9318 - loss: 0.1752 - val_accuracy: 0.9358 - val_loss: 0.1610
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9370 - loss: 0.1604 - val_accuracy: 0.9384 - val_loss: 0.1552
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9404 - loss: 0.1530 - val_accuracy: 0.9400 - val_loss: 0.1510
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9431 - loss: 0.1461 - val_accuracy: 0.9417 - val_loss: 0.1476
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9444 - loss: 0.1433 - val_accuracy: 0.9419 - val_loss: 0.1477
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9460 - loss: 0.1386 - val_accuracy: 0.9417 - val_loss: 0.1483
Epoch 8/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 

2024/09/22 18:21:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 18:21:28 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 18:21:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run defiant-ape-959 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/25c22f52e35347ecb89e47e6ceee9b16.
2024/09/22 18:21:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


Model saved as Bidirectional_LSTM_1000_16 with accuracy: 0.94
Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.8969 - loss: 0.2613 - val_accuracy: 0.9341 - val_loss: 0.1660
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9350 - loss: 0.1647 - val_accuracy: 0.9386 - val_loss: 0.1524
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9416 - loss: 0.1486 - val_accuracy: 0.9409 - val_loss: 0.1488
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9453 - loss: 0.1390 - val_accuracy: 0.9418 - val_loss: 0.1465
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9483 - loss: 0.1328 - val_accuracy: 0.9429 - val_loss: 0.1437
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9505 - loss: 0.1282 - val_accuracy: 0.9449 - val_loss: 0.1444
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9528 - loss: 0.1226 - val_accuracy: 0.9459 - val_loss: 0.1391
Epoch 8

2024/09/22 18:26:09 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 18:26:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 18:26:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run likeable-snake-443 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/42e5bb57663847d1984a7c5848ca5897.
2024/09/22 18:26:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


Model saved as Bidirectional_LSTM_1000_32 with accuracy: 0.95
Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 54s 5ms/step - accuracy: 0.9047 - loss: 0.2403 - val_accuracy: 0.9366 - val_loss: 0.1585
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9392 - loss: 0.1533 - val_accuracy: 0.9426 - val_loss: 0.1458
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9449 - loss: 0.1393 - val_accuracy: 0.9437 - val_loss: 0.1418
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 51s 5ms/step - accuracy: 0.9495 - loss: 0.1301 - val_accuracy: 0.9448 - val_loss: 0.1405
Epoch 5/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9532 - loss: 0.1209 - val_accuracy: 0.9467 - val_loss: 0.1366
Epoch 6/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9563 - loss: 0.1138 - val_accuracy: 0.9487 - val_loss: 0.1356
Epoch 7/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9591 - loss: 0.1061 - val_accuracy: 0.9492 - val_loss: 0.1369
Epoch 8

2024/09/22 18:34:46 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 18:34:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/22 18:34:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-cow-286 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/e4352c9a0f1f4b169800149bd3620954.
2024/09/22 18:34:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


Model saved as Bidirectional_LSTM_1000_64 with accuracy: 0.95
Epoch 1/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 131s 12ms/step - accuracy: 0.9084 - loss: 0.2295 - val_accuracy: 0.9386 - val_loss: 0.1547
Epoch 2/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 135s 13ms/step - accuracy: 0.9404 - loss: 0.1511 - val_accuracy: 0.9430 - val_loss: 0.1444
Epoch 3/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 125s 12ms/step - accuracy: 0.9487 - loss: 0.1318 - val_accuracy: 0.9471 - val_loss: 0.1349
Epoch 4/10
10373/10373 ━━━━━━━━━━━━━━━━━━━━ 135s 13ms/step - accuracy: 0.9532 - loss: 0.1205 - val_accuracy: 0.9471 - val_loss: 0.1357
Epoch 5/10
10116/10373 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9583 - loss: 0.1080

2024/09/22 18:45:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run gentle-rook-187 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/4ae09ce00629496fb41f0fe627bce2e6.
2024/09/22 18:45:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


KeyboardInterrupt: 

# Chosen Model 
## Bi-directional LSTM
### Params
- input_dim : 1000
- output_dim : 128
- input_length : 10

In [9]:
import os
import mlflow
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Define directories and parameters
save_dir = "model_token"
os.makedirs(save_dir, exist_ok=True)
input_dim = 1000  # Adjust as needed
output_dim = 64  # Adjust as needed
input_length = 10  # Should match the training setup

# Sample data (assuming filtered_questions and filtered_labels are already defined)
# filtered_questions = [...]  # Define your questions list here
# filtered_labels = [...]     # Define your labels list here

# Tokenization and Padding
tokenizer = Tokenizer(num_words=input_dim)
tokenizer.fit_on_texts(filtered_questions)
sequences = tokenizer.texts_to_sequences(filtered_questions)
X = pad_sequences(sequences, maxlen=input_length)
y = np.array(filtered_labels.tolist())

# Save the tokenizer
tokenizer_path = os.path.join(save_dir, "tokenizer.pkl")
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)
print(f"Tokenizer saved at: {tokenizer_path}")

# Split data for training and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Start MLflow run
with mlflow.start_run():
    # Model: Simple LSTM
    model_name = f"Simple_LSTM_{input_dim}_{output_dim}"
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
        LSTM(output_dim),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with a validation split to monitor training
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

    # Save the model in the correct format
    model_path = os.path.join(save_dir, model_name + ".keras")
    model.save(model_path)
    print(f"Model saved at: {model_path}")

    # Log the model to MLflow
    mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
    mlflow.keras.log_model(model, artifact_path=model_name)
    print(f"Model logged in MLflow as {model_name}")

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Model evaluation - Accuracy: {accuracy:.2f}, Loss: {loss:.2f}")

# End the MLflow run
mlflow.end_run()


Tokenizer saved at: model_token/tokenizer.pkl
Epoch 1/10
10373/10373 - 30s - 3ms/step - accuracy: 0.9187 - loss: 0.2055 - val_accuracy: 0.9347 - val_loss: 0.1668
Epoch 2/10
10373/10373 - 29s - 3ms/step - accuracy: 0.9362 - loss: 0.1604 - val_accuracy: 0.9392 - val_loss: 0.1516
Epoch 3/10
10373/10373 - 30s - 3ms/step - accuracy: 0.9422 - loss: 0.1467 - val_accuracy: 0.9427 - val_loss: 0.1448
Epoch 4/10
10373/10373 - 30s - 3ms/step - accuracy: 0.9462 - loss: 0.1367 - val_accuracy: 0.9449 - val_loss: 0.1405
Epoch 5/10
10373/10373 - 29s - 3ms/step - accuracy: 0.9497 - loss: 0.1293 - val_accuracy: 0.9461 - val_loss: 0.1379
Epoch 6/10
10373/10373 - 29s - 3ms/step - accuracy: 0.9522 - loss: 0.1220 - val_accuracy: 0.9449 - val_loss: 0.1377
Epoch 7/10
10373/10373 - 28s - 3ms/step - accuracy: 0.9554 - loss: 0.1163 - val_accuracy: 0.9466 - val_loss: 0.1403
Epoch 8/10
10373/10373 - 28s - 3ms/step - accuracy: 0.9572 - loss: 0.1111 - val_accuracy: 0.9474 - val_loss: 0.1412
Epoch 9/10
10373/10373 - 3

2024/09/24 09:36:20 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Model saved at: model_token/Simple_LSTM_1000_64.keras


/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/09/24 09:37:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run invincible-tern-528 at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1/runs/ae526ecd037b43e69936e94473d00f29.
2024/09/24 09:37:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-1073438601911.us-west2.run.app/#/experiments/1.


MlflowException: API request to https://mlflow-service-1073438601911.us-west2.run.app/api/2.0/mlflow-artifacts/artifacts/1/ae526ecd037b43e69936e94473d00f29/artifacts/Simple_LSTM_1000_64/python_env.yaml failed with exception HTTPSConnectionPool(host='mlflow-service-1073438601911.us-west2.run.app', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/1/ae526ecd037b43e69936e94473d00f29/artifacts/Simple_LSTM_1000_64/python_env.yaml (Caused by ResponseError('too many 500 error responses'))